<a href="https://colab.research.google.com/github/dev-corvus/imersao_alura_IA/blob/main/caltrackia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U google-genai

In [2]:
import google.generativeai as genai
import re
import os
from google.colab import userdata
from datetime import date # Import the date

# Get the API key from Google Colab user data
# Make sure you have stored your API key under the name 'GOOGLE_API_KEY' in Colab Secrets
api_key = userdata.get('GOOGLE_API_KEY')

# Configure the generative AI library with the retrieved API key
genai.configure(api_key=api_key)

# Initialize the Gemini model
model = genai.GenerativeModel('gemini-2.0-flash')

def get_macro_estimate(food_description):
    """Sends a prompt to Gemini to estimate macros for a food description."""
    prompt = f"Estimate the protein, carbohydrates, fat, fiber and sugar content for: {food_description}. Provide the answer in grams.  Return the data in the format 'Protein: Xg, Carbohydrates: Yg, Fat: Zg, Fiber: Ag, Sugar: Bg'"
    try:
      response = model.generate_content(prompt)
      return response.text
    except Exception as e:
        return f"Error generating content: {e}"

def extract_macros(text):
    """
    Extracts protein, carbs, and fat values from the Gemini response string.
    Handles potential variations in the string format.
    """
    macros = {'protein': None, 'carbs': None, 'fat': None, 'fiber': None, 'sugar': None}
    # Use regular expressions to find the numeric values following "Protein:", "Carbohydrates:", and "Fat:"
    protein_match = re.search(r"Protein:\s*(\d+\.?\d*)g", text, re.IGNORECASE)
    carbs_match = re.search(r"Carbohydrates?:\s*(\d+\.?\d*)g", text, re.IGNORECASE)  #Handles Carbohydrates or Carbohydrate
    fat_match = re.search(r"Fat:\s*(\d+\.?\d*)g", text, re.IGNORECASE)
    fiber_match = re.search(r"Fiber:\s*(\d+\.?\d*)g", text, re.IGNORECASE)
    sugar_match = re.search(r"Sugar:\s*(\d+\.?\d*)g", text, re.IGNORECASE)


    if protein_match:
        macros['protein'] = float(protein_match.group(1))
    if carbs_match:
        macros['carbs'] = float(carbs_match.group(1))
    if fat_match:
        macros['fat'] = float(fat_match.group(1))
    if fiber_match:
        macros['fiber'] = float(fiber_match.group(1))
    if sugar_match:
        macros['sugar'] = float(sugar_match.group(1))
    return macros

def log_meal():
    """Logs a meal and returns the estimated macros along with the food description and meal type."""
    food_input = input("What did you eat? (e.g., 100g grilled chicken, a banana): ")

    # Ask for meal type
    meal_type = ""
    valid_meal_types = ['breakfast', 'lunch', 'snack', 'dinner']
    while meal_type not in valid_meal_types:
        meal_type = input("What meal was this? (breakfast, lunch, snack, dinner): ").lower()
        if meal_type not in valid_meal_types:
            print("Invalid meal type. Please enter 'breakfast', 'lunch', 'snack', or 'dinner'.")


    gemini_response = get_macro_estimate(food_input)
    print(f"Gemini's estimate: {gemini_response}")

    macros = extract_macros(gemini_response)
    if all(value is not None for value in macros.values()): # Check if all macros were extracted
        print(f"Extracted macros: Protein: {macros['protein']:.2f}g, Carbs: {macros['carbs']:.2f}g, Fat: {macros['fat']:.2f}g, Fiber: {macros['fiber']:.2f}g, Sugar: {macros['sugar']:.2f}g")
        # Return the food description, extracted macros, and meal type
        return {'food': food_input, 'macros': macros, 'type': meal_type}
    else:
        print("Error: Could not extract all macro values.  Returning empty")
        return {}


def display_summary(daily_log):
    """Displays the daily macro summary categorized by meal type."""
    if not daily_log:
        print("No meals logged today.")
        return

    today = str(date.today())
    print(f"\n--- Daily Macro Summary for {today} ---")

    # Initialize total macros for each meal type
    meal_type_totals = {
        'breakfast': {'protein': 0, 'carbs': 0, 'fat': 0, 'fiber': 0, 'sugar': 0},
        'lunch': {'protein': 0, 'carbs': 0, 'fat': 0, 'fiber': 0, 'sugar': 0},
        'snack': {'protein': 0, 'carbs': 0, 'fat': 0, 'fiber': 0, 'sugar': 0},
        'dinner': {'protein': 0, 'carbs': 0, 'fat': 0, 'fiber': 0, 'sugar': 0},
        'total': {'protein': 0, 'carbs': 0, 'fat': 0, 'fiber': 0, 'sugar': 0} # Overall total
    }

    meal_details_by_type = {
        'breakfast': [],
        'lunch': [],
        'snack': [],
        'dinner': []
    }

    for meal in daily_log:
        # Check if the meal dictionary has 'date' and 'type' keys before accessing
        if 'date' in meal and meal['date'] == today and 'type' in meal and meal['type'] in meal_type_totals:
            meal_type = meal['type']
            macros = meal['macros']

            # Accumulate macros for the specific meal type
            meal_type_totals[meal_type]['protein'] += macros.get('protein', 0)
            meal_type_totals[meal_type]['carbs'] += macros.get('carbs', 0)
            meal_type_totals[meal_type]['fat'] += macros.get('fat', 0)
            meal_type_totals[meal_type]['fiber'] += macros.get('fiber', 0)
            meal_type_totals[meal_type]['sugar'] += macros.get('sugar', 0)

            # Accumulate overall total macros
            meal_type_totals['total']['protein'] += macros.get('protein', 0)
            meal_type_totals['total']['carbs'] += macros.get('carbs', 0)
            meal_type_totals['total']['fat'] += macros.get('fat', 0)
            meal_type_totals['total']['fiber'] += macros.get('fiber', 0)
            meal_type_totals['total']['sugar'] += macros.get('sugar', 0)

            # Store meal details for later display
            meal_details_by_type[meal_type].append(meal)


    # Display summary by meal type
    for meal_type in ['breakfast', 'lunch', 'snack', 'dinner']:
        print(f"\n--- {meal_type.capitalize()} Summary ---")
        print(f"Total Protein: {meal_type_totals[meal_type]['protein']:.2f}g")
        print(f"Total Carbohydrates: {meal_type_totals[meal_type]['carbs']:.2f}g")
        print(f"Total Fat: {meal_type_totals[meal_type]['fat']:.2f}g")
        print(f"Total Fiber: {meal_type_totals[meal_type]['fiber']:.2f}g")
        print(f"Total Sugar: {meal_type_totals[meal_type]['sugar']:.2f}g")

        # Display meal details for this type
        if meal_details_by_type[meal_type]:
            print(f"  {meal_type.capitalize()} Details:")
            for meal in meal_details_by_type[meal_type]:
                 print(f"    {meal['food']}:")
                 print(f"      Protein: {meal['macros'].get('protein', 0):.2f}g, Carbs: {meal['macros'].get('carbs', 0):.2f}g, Fat: {meal['macros'].get('fat', 0):.2f}g, Fiber: {meal['macros'].get('fiber', 0):.2f}g, Sugar: {meal['macros'].get('sugar', 0):.2f}g")


    # Display overall daily total
    print("\n--- Overall Daily Total ---")
    print(f"Total Protein: {meal_type_totals['total']['protein']:.2f}g")
    print(f"Total Carbohydrates: {meal_type_totals['total']['carbs']:.2f}g")
    print(f"Total Fat: {meal_type_totals['total']['fat']:.2f}g")
    print(f"Total Fiber: {meal_type_totals['total']['fiber']:.2f}g")
    print(f"Total Sugar: {meal_type_totals['total']['sugar']:.2f}g")


# Main loop
daily_log = [] # Renamed for clarity
while True:
    action = input("Enter 'log' to record a meal, 'summary' to see your total, or 'quit' to exit: ").lower()
    if action == 'log':
        meal_data = log_meal() # Capture both food and macros
        if meal_data: # only append if meal_data is not empty
            meal_data['date'] = str(date.today()) # Add the current date to the meal data
            daily_log.append(meal_data)
    elif action == 'summary':
        display_summary(daily_log)
    elif action == 'quit':
        break
    else:
        print("Invalid action. Please enter 'log', 'summary', or 'quit'.")

print("Thank you for tracking your macros!")

Enter 'log' to record a meal, 'summary' to see your total, or 'quit' to exit: log
What did you eat? (e.g., 100g grilled chicken, a banana): chicken beef
What meal was this? (breakfast, lunch, snack, dinner): lunch
Gemini's estimate: Okay, I will provide estimates based on typical serving sizes and average composition. Keep in mind that these are approximations, as the exact values depend on the cut of meat, cooking method, and portion size. I'll assume a 100g (3.5 oz) serving for each.

**Chicken (100g, Skinless Breast, Cooked):**

Protein: 31g, Carbohydrates: 0g, Fat: 3.6g, Fiber: 0g, Sugar: 0g

**Beef (100g, Lean Ground Beef, Cooked):**

Protein: 26g, Carbohydrates: 0g, Fat: 15g, Fiber: 0g, Sugar: 0g

Extracted macros: Protein: 31.00g, Carbs: 0.00g, Fat: 3.60g, Fiber: 0.00g, Sugar: 0.00g
Enter 'log' to record a meal, 'summary' to see your total, or 'quit' to exit: summary

--- Daily Macro Summary for 2025-05-18 ---

--- Breakfast Summary ---
Total Protein: 0.00g
Total Carbohydrates: 

KeyboardInterrupt: Interrupted by user